In [3]:
cd ../..

D:\Users\mrefoyo\Proyectos\Sub-SpaCE_plus


In [4]:
import os
import pickle
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf

from experiments.evaluation.evaluation_utils import load_dataset_for_eval, calculate_metrics_for_dataset

print(tf.__version__)

2.13.0


In [9]:
# DATASETS = ['CBF', 'chinatown', 'coffee', 'gunpoint', 'ECG200']
DATASETS = ["BasicMotions", "NATOPS", "UWaveGestureLibrary"]
model_to_explain = "cls_basic_train"
ae_name = "ae_basic_train"
methods = {
    # Basic experimentos
    # "43b17a2e4f21097540adf8600402c5bd32fa60d6": "subspace_gknn_gch",
    # "37a3f06c077321b95960769282152427a7ee48ae": "subspace_gknn_ich", 
    # "86df14d56b66fecfb65d5805107dfe10961ecae9": "subspace_iknn_gch",     
    # "4f40e39bfe0d3da692b5571d87f7ec915c42eba2": "subspace_iknn_ich", 
    
    # Basic experiments only gknn but with lower gamma (0.125)
    # "4d1faa458bab2b8c6ee14d59b824557fcc80d8e3": "subspace_gknn_gch_gamma0125", 
    # "93c0b5f0630a5dbf5d0377a4f3ff03b6868cbda1": "subspace_iknn_gch_gamma0125", 
    
    # Basic experiments New IoS 
    # "e54dcb25b02132166a719885cad2695540450ee6": "subspace_gknn_gch_ios",
    # "9cdb2fb25d7c664d60da7aa23b4c2c0f9ec9548c": "subspace_gknn_ich_ios",
    # "8c29354f168b9852a05523a7c26ab0f4acb30e9b": "subspace_iknn_gch_ios",
    # "b93fe63854fe36f62900cbcae1c00ad499d41f74": "subspace_iknn_ich_ios",
    
    # Sub-SpaCE v2 para fine tuning con optimizacion con canales independientes
    # "316686353c28547589d494bcc886c8334329b14e": "subspace_gknn_gch",
    # "0f31693cb3d17afd3d67191e82e2bca887075ee6": "subspace_gknn_gch_moremut",
    # "76f8a8271b7ba6d171b7383bce7bbf4d9286d78c": "subspace_gknn_gch_moreit",
    # "bba03e82f6c718e1466a06087dd99606de229b40": "subspace_gknn_ich",
    # "a246def090b506ff8ba41e861e0464493233b9d1": "subspace_gknn_ich_moremut",
    # "d3b12b6ac5e2ab4e68901859273a55edb14d5590": "subspace_gknn_ich_moreit",
    # "cfbb6793595875fa49b7becafefa2ff2df33a2f6": "subspace_gknn_cch",
    # "e4a8da55a30853287350d3540873bcb91cb50b70": "subspace_gknn_cch_moremut",
    # "fe0be96f08c5d37950f7ae2c50d7f81fc050c185": "subspace_gknn_cch_moreit",
    
    # Sub-SpaCE v2 con NUN sinteticos
    # "0f31693cb3d17afd3d67191e82e2bca887075ee6": "subspace_gknn_gch_moremut",
    # "988e9ee7e36478e6b02628f6fd15e8f6fdabce63": "subspace_iknn_gch_moremut",
    # "a246def090b506ff8ba41e861e0464493233b9d1": "subspace_gknn_ich_moremut",
    # "de59140e133b8214d6fd39630d49420f41f2f395": "subspace_iknn_ich_moremut",
    # "e4a8da55a30853287350d3540873bcb91cb50b70": "subspace_gknn_cch_moremut",
    # "af3bb75a4e9d84bc4cb545b21941e6750ce17b45": "subspace_iknn_cch_moremut",
    
    # Sub-SpaCE v2 mutation study
    # "a39f776a35cfd7c4530ed4c8f104fcef9b650512": "subspace_gknn_ich_mut01",
    # "99b876d673ecb07aa65860859a2f6467617b8c86": "subspace_gknn_gch_mut01",
    # "7469da9fdf840b4b4c2b38bc4bc9111016b32de5": "subspace_gknn_cch_mut01",
    # "db736bc9c4068ecbfbc96a90e98dc0eb3efbaa92": "subspace_gknn_ich_mut02",
    # "cbbcedcb6d9fd15f864bcc72209e990664f578cc": "subspace_gknn_gch_mut02",
    # "3a2f5d028e5e5ee1637634010ddab48db711879f": "subspace_gknn_cch_mut02",
    # "e702bb244e58da0e7ebbc277bb04c45fb09118aa": "subspace_gknn_ich_mut04",
    # "5643e1c660ae69f41f58f9b71109809ac41d806a": "subspace_gknn_gch_mut04",
    # "d3103c82728b7e2d32be72d2c3029c225869b952": "subspace_gknn_cch_mut04",
    # "6a1225fc4657ca27a9d23ed21050b2fb890175c3": "subspace_gknn_ich_mut05",
    # "41f42efc9de796a8dafc08265dc029d606e452cf": "subspace_gknn_gch_mut05",
    # "2a7b2e8d4df37d7d7ece4e9b45999a3679252362": "subspace_gknn_cch_mut05",
    # "14f1a162dc03f18a16cb1044efd076f3a4fb61a2": "subspace_gknn_ich_mut07",
    # "e76bb3c93b88f711dd5d25a3ac5704c3bf98b4cf": "subspace_gknn_gch_mut07",
    # "e58d4a60e2c54feed0add715c4c3b28afbb37969": "subspace_gknn_cch_mut07",
    # "6d734428c0fea87d3327bea1408b47c483d33e9e": "subspace_gknn_ich_mut09",
    # "e347534c7eb328d5ddafe940f724a02f9a14ec80": "subspace_gknn_gch_mut09",
    # "b945f3452ca7deb539f84c45d6c6202e75563c55": "subspace_gknn_cch_mut09",
    
    # First comparison with other methods
    "comte": "comte",
    "abcf": "abcf",
    "discox": "discox",
    "d455a715713b180b88739f05c8ef8ea313d6abfc": "subspace_gknn_cch_mut07",
    
}



# Get results

In [10]:
# Data and aux data dict
data_dict = {}
models_dict = {}
outlier_calculators_dict = {}
possible_nuns_dict = {}
desired_classes_dict = {}
original_classes_dict = {}

# Results dicts
mean_results_dict = {}
methods_cfs_dict = {}
results_all_datasets_df = pd.DataFrame()
common_test_indexes_dict = {}

# Iterate through datasets
for dataset in DATASETS:
    print(f'Calculating metrics for {dataset}')
    # Load all info needed to get the counterfactual
    data_tuple, original_classes, model, outlier_calculator, possible_nuns, desired_classes = load_dataset_for_eval(dataset, model_to_explain, ae_name)
    data_dict[dataset] = data_tuple
    models_dict[dataset] = model
    outlier_calculators_dict[dataset] = outlier_calculator
    possible_nuns_dict[dataset] = possible_nuns
    desired_classes_dict[dataset] = desired_classes
    original_classes_dict[dataset] = original_classes

    # Get the metrics for all methods
    dataset_mean_std_df, dataset_results_df, method_cfs_dataset, common_test_indexes = calculate_metrics_for_dataset(
        dataset, methods, model_to_explain,
        data_tuple, original_classes, model, outlier_calculator, possible_nuns
    )
    mean_results_dict[dataset] = dataset_mean_std_df
    methods_cfs_dict[dataset] = method_cfs_dataset
    results_all_datasets_df = pd.concat([results_all_datasets_df, dataset_results_df])
    common_test_indexes_dict[dataset] = common_test_indexes

    # Store results
    dataset_mean_std_df.to_csv(f'./experiments/evaluation/results_mean_metrics_{dataset}.csv', sep=";", index=False)

# Store all results
results_all_datasets_df.to_csv(f'./experiments/evaluation/results_all.csv', sep=";", index=False)


Calculating metrics for BasicMotions
abcf


100%|██████████| 40/40 [00:01<00:00, 20.08it/s]


comte


100%|██████████| 40/40 [00:01<00:00, 21.02it/s]


d455a715713b180b88739f05c8ef8ea313d6abfc


...

100%|██████████| 100/100 [00:04<00:00, 20.85it/s]


d455a715713b180b88739f05c8ef8ea313d6abfc


100%|██████████| 100/100 [00:04<00:00, 20.64it/s]


discox


100%|██████████| 100/100 [00:05<00:00, 19.78it/s]


In [11]:
from IPython.display import display
for dataset in DATASETS:
    print(dataset)
    display(mean_results_dict[dataset])

BasicMotions


,method,ii,nchanges,sparsity,L1,L2,proba,valid,nuns_valid,outlier_score,increase_outlier_score,subsequences,subsequences %,times
0,abcf,19.5 ± 11.69,328.78 ± 170.0,0.55 ± 0.28,1257.44 ± 1157.54,89.51 ± 67.4,0.51 ± 0.11,0.68 ± 0.47,nan ± nan,0.4 ± 0.24,0.03 ± 0.03,10.48 ± 6.38,0.03 ± 0.02,0.81 ± 0.75
1,comte,19.5 ± 11.69,287.42 ± 46.32,0.48 ± 0.08,1353.76 ± 836.66,103.36 ± 60.47,0.87 ± 0.12,1.0 ± 0.0,nan ± nan,0.49 ± 0.32,0.14 ± 0.17,1.55 ± 0.6,0.01 ± 0.0,1138.65 ± 9.95
2,subspace_gknn_cch_mut07,19.5 ± 11.69,270.98 ± 148.86,0.45 ± 0.25,960.68 ± 961.2,73.8 ± 57.13,0.7 ± 0.14,1.0 ± 0.0,1.0 ± 0.0,0.24 ± 0.16,0.01 ± 0.02,8.75 ± 3.72,0.03 ± 0.01,35.89 ± 9.73
3,discox,19.5 ± 11.69,78.75 ± 25.19,0.13 ± 0.04,460.58 ± 206.72,68.75 ± 23.75,0.34 ± 0.07,0.6 ± 0.5,nan ± nan,0.43 ± 0.14,0.19 ± 0.11,25.88 ± 18.02,0.09 ± 0.06,470.68 ± 151.86


NATOPS


,method,ii,nchanges,sparsity,L1,L2,proba,valid,nuns_valid,outlier_score,increase_outlier_score,subsequences,subsequences %,times
0,abcf,87.18 ± 53.72,919.53 ± 230.35,0.75 ± 0.19,307.74 ± 137.75,14.82 ± 6.05,0.69 ± 0.18,0.86 ± 0.35,nan ± nan,0.62 ± 0.19,0.09 ± 0.11,53.02 ± 19.52,0.09 ± 0.03,0.98 ± 0.68
1,comte,87.18 ± 53.72,166.86 ± 53.52,0.14 ± 0.04,78.53 ± 60.13,7.87 ± 4.78,0.73 ± 0.16,0.92 ± 0.27,nan ± nan,0.79 ± 0.33,0.24 ± 0.19,2.96 ± 0.82,0.0 ± 0.0,1458.82 ± 192.98
2,subspace_gknn_cch_mut07,87.18 ± 53.72,358.69 ± 150.2,0.29 ± 0.12,162.87 ± 121.65,10.75 ± 6.46,0.93 ± 0.06,1.0 ± 0.0,1.0 ± 0.0,0.57 ± 0.18,0.04 ± 0.04,23.64 ± 2.71,0.04 ± 0.0,34.17 ± 9.04
3,discox,87.18 ± 53.72,85.12 ± 24.58,0.07 ± 0.02,29.94 ± 24.86,6.2 ± 4.04,0.4 ± 0.09,0.17 ± 0.38,nan ± nan,0.71 ± 0.35,0.18 ± 0.17,52.06 ± 20.02,0.09 ± 0.03,1329.99 ± 261.59


UWaveGestureLibrary


,method,ii,nchanges,sparsity,L1,L2,proba,valid,nuns_valid,outlier_score,increase_outlier_score,subsequences,subsequences %,times
0,abcf,159.0 ± 87.23,406.99 ± 205.66,0.43 ± 0.22,393.43 ± 196.33,23.39 ± 7.64,0.49 ± 0.16,0.93 ± 0.26,nan ± nan,0.72 ± 0.18,0.2 ± 0.16,6.33 ± 2.91,0.01 ± 0.01,0.8 ± 0.69
1,comte,159.0 ± 87.23,828.44 ± 171.39,0.88 ± 0.18,707.25 ± 200.53,30.68 ± 6.67,0.97 ± 0.02,1.0 ± 0.0,nan ± nan,0.45 ± 0.09,0.03 ± 0.06,1.07 ± 0.26,0.0 ± 0.0,1463.74 ± 194.95
2,subspace_gknn_cch_mut07,159.0 ± 87.23,295.69 ± 157.79,0.31 ± 0.17,279.09 ± 144.05,18.9 ± 5.92,0.81 ± 0.12,1.0 ± 0.0,1.0 ± 0.0,0.58 ± 0.13,0.07 ± 0.07,5.39 ± 2.37,0.01 ± 0.01,57.33 ± 16.28
3,discox,159.0 ± 87.23,216.92 ± 83.14,0.23 ± 0.09,185.59 ± 119.82,15.63 ± 7.48,0.33 ± 0.08,0.39 ± 0.49,nan ± nan,0.72 ± 0.21,0.21 ± 0.19,29.08 ± 25.62,0.06 ± 0.05,4901.37 ± 2014.46


In [12]:
columns = ["nchanges", "L2", "valid", "increase_outlier_score", "subsequences", "times"]
for dataset in DATASETS:
    print(dataset)
    display(mean_results_dict[dataset].set_index("method").sort_index()[columns])

BasicMotions


,nchanges,L2,valid,increase_outlier_score,subsequences,times
method,,,,,,
abcf,328.78 ± 170.0,89.51 ± 67.4,0.68 ± 0.47,0.03 ± 0.03,10.48 ± 6.38,0.81 ± 0.75
comte,287.42 ± 46.32,103.36 ± 60.47,1.0 ± 0.0,0.14 ± 0.17,1.55 ± 0.6,1138.65 ± 9.95
discox,78.75 ± 25.19,68.75 ± 23.75,0.6 ± 0.5,0.19 ± 0.11,25.88 ± 18.02,470.68 ± 151.86
subspace_gknn_cch_mut07,270.98 ± 148.86,73.8 ± 57.13,1.0 ± 0.0,0.01 ± 0.02,8.75 ± 3.72,35.89 ± 9.73


NATOPS


,nchanges,L2,valid,increase_outlier_score,subsequences,times
method,,,,,,
abcf,919.53 ± 230.35,14.82 ± 6.05,0.86 ± 0.35,0.09 ± 0.11,53.02 ± 19.52,0.98 ± 0.68
comte,166.86 ± 53.52,7.87 ± 4.78,0.92 ± 0.27,0.24 ± 0.19,2.96 ± 0.82,1458.82 ± 192.98
discox,85.12 ± 24.58,6.2 ± 4.04,0.17 ± 0.38,0.18 ± 0.17,52.06 ± 20.02,1329.99 ± 261.59
subspace_gknn_cch_mut07,358.69 ± 150.2,10.75 ± 6.46,1.0 ± 0.0,0.04 ± 0.04,23.64 ± 2.71,34.17 ± 9.04


UWaveGestureLibrary


,nchanges,L2,valid,increase_outlier_score,subsequences,times
method,,,,,,
abcf,406.99 ± 205.66,23.39 ± 7.64,0.93 ± 0.26,0.2 ± 0.16,6.33 ± 2.91,0.8 ± 0.69
comte,828.44 ± 171.39,30.68 ± 6.67,1.0 ± 0.0,0.03 ± 0.06,1.07 ± 0.26,1463.74 ± 194.95
discox,216.92 ± 83.14,15.63 ± 7.48,0.39 ± 0.49,0.21 ± 0.19,29.08 ± 25.62,4901.37 ± 2014.46
subspace_gknn_cch_mut07,295.69 ± 157.79,18.9 ± 5.92,1.0 ± 0.0,0.07 ± 0.07,5.39 ± 2.37,57.33 ± 16.28


# Counterfactual visualization

In [10]:
import math
def plot_comp_cfs(x_orig, possible_nuns, x_cfs_dict):
    
    # Calculate auxiliary variables
    method_names = list(x_cfs_dict.keys())
    x_cfs = list(x_cfs_dict.values())
    length = x_orig.shape[0]
    n_channels = x_orig.shape[1]

    # Define outer grid
    fig = plt.figure(figsize=(22, 16))
    grid_columns = 2
    grid_rows = math.ceil(len(x_cfs) / grid_columns)
    outer_grid = gridspec.GridSpec(grid_rows, grid_columns, wspace=0.1, hspace=0.3)

    # Iterate through outer grid plots
    for m, outer in enumerate(outer_grid):
        try:
            # Get change mask 
            x_cf = x_cfs[m]
            method_name = method_names[m]
            # Get nuns used by the method depending on the name
            if "gknn" in method_name:
                nun_example = possible_nuns["gknn"]
            elif "iknn" in method_name:
                nun_example = possible_nuns["iknn"]
            else:
                raise ValueError('Not detected NUN finding procedure in name. Method name must contain "gknn" or "iknn"')
            found_counterfactual_mask = (x_orig != x_cf.reshape(1, length, n_channels)).astype(int)
            diff_mask = np.diff(found_counterfactual_mask, prepend=0, append=0, axis=1)
        except IndexError:
            break
    
        # Create inner grid
        inner_grid = gridspec.GridSpecFromSubplotSpec(n_channels, 1, wspace=0.2, hspace=0.2, subplot_spec=outer)
        sub_channel_axs = []
        for i in range(n_channels):
            ax = plt.Subplot(fig, inner_grid[i])
            ax.plot(nun_example.reshape(1, length, n_channels)[:, :, i].flatten(), color='grey')
            ax.plot(x_cf.reshape(1, length, n_channels)[:, :, i].flatten(), color='red')
            ax.plot(x_orig.reshape(1, length, n_channels)[:, :, i].flatten(), color="#332288")
            # Set title if first channel
            if i == 0:
                ax.set_title(method_name)
            
            # Add fill to changes
            starts = np.clip( np.where(diff_mask[0, :, i].flatten() == 1)[0] - 1 , 0, length)
            ends = np.where(diff_mask[0, :, i].flatten() == -1)[0]
            # Iterate over the mask segments and fill them with red color
            for i in range(0, len(starts)):
                start_idx = starts[i]
                end_idx = ends[i]
                # plt.fill_between(comp_df.iloc[start_idx:end_idx].index, comp_df.min(), comp_df.max(), color='red', alpha=0.7)
                ax.axvspan(start_idx, end_idx, alpha=0.2, color='red')
            
            # Add ax to list of subplots
            fig.add_subplot(ax)
            sub_channel_axs.append(ax)

    # Plot figure
    fig.legend(
        sub_channel_axs,     # The line objects
        labels=['NUN', 'x_cf', 'x_orig'],   # The labels for each line
        loc='center',
        prop={'size': 12}
    )
    plt.show()



In [11]:
dataset = "UWaveGestureLibrary"
results_all_datasets_df[results_all_datasets_df["dataset"]==dataset]

,ii,nchanges,sparsity,L1,L2,proba,valid,nuns_valid,outlier_score,increase_outlier_score,subsequences,subsequences %,times,method,order,dataset
0,3,630.0,0.666667,849.672028,40.468249,0.994502,True,NaN,0.357573,0.000000,1.0,0.002116,1291.526145,comte,1,UWaveGestureLibrary
1,13,945.0,1.000000,701.469408,29.498525,0.986486,True,NaN,0.394286,0.000000,1.0,0.002116,1301.269161,comte,1,UWaveGestureLibrary
2,14,630.0,0.666667,534.941475,27.801814,0.981622,True,NaN,0.429190,0.000000,1.0,0.002116,1301.799412,comte,1,UWaveGestureLibrary
3,15,945.0,1.000000,753.060521,31.011630,0.989560,True,NaN,0.375455,0.000000,1.0,0.002116,1309.426037,comte,1,UWaveGestureLibrary
4,19,630.0,0.666667,546.554781,26.437987,0.989558,True,NaN,0.478830,0.000000,1.0,0.002116,1316.859315,comte,1,UWaveGestureLibrary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,300,158.0,0.167196,144.647351,12.668673,0.772654,True,1.0,0.487605,0.041953,3.0,0.006349,48.047095,subspace_gknn_cch_mut07,2,UWaveGestureLibrary
96,302,84.0,0.088889,82.245441,9.598369,0.449715,True,1.0,0.591355,0.123180,5.0,0.010582,44.687890,subspace_gknn_cch_mut07,2,UWaveGestureLibrary
97,303,43.0,0.045503,38.132580,6.282867,0.620453,True,1.0,0.525460,0.036501,2.0,0.004233,43.703472,subspace_gknn_cch_mut07,2,UWaveGestureLibrary
98,314,570.0,0.603175,556.326838,27.585234,0.814680,True,1.0,0.667473,0.013069,6.0,0.012698,83.094417,subspace_gknn_cch_mut07,2,UWaveGestureLibrary


In [12]:
# methods = ["subspace_grouped", "subspace_v2_grouped", "subspace_individual", "subspace_v2_individual"]
common_index_instance = 13

In [13]:
# Get data
X_train, y_train, X_test, y_test = data_dict[dataset]
common_indexes = common_test_indexes_dict[dataset]
instance_index = common_indexes[common_index_instance]

# Get common cfs calculated for all method
x_orig = X_test[instance_index]

# Get possible NUNs
possible_nuns = possible_nuns_dict[dataset]
possible_nuns_instance = {type: possible_nuns[type][instance_index] for type in ["gknn", "iknn"]}

# Get original class and desired class
original_class = original_classes_dict[dataset][instance_index]
desired_class = desired_classes_dict[dataset][instance_index]

# Get the cf calculated for all methods
sol_cfs_methods = methods_cfs_dict[dataset]
sol_cfs_methods = {method: sol_cfs[common_index_instance]['cf'] for method, sol_cfs in sol_cfs_methods.items()}

# Show result
print(f'Sample {instance_index}: true class {y_test[instance_index]}, x_orig predicted {original_class}, desired_class  {desired_class}')
plot_comp_cfs(x_orig, possible_nuns_instance, sol_cfs_methods)


Sample 51: true class 1, x_orig predicted 1, desired_class  6


ValueError: Not detected NUN finding procedure in name. Method name must contain "gknn" or "iknn"

<Figure size 2200x1600 with 0 Axes>

In [ ]:
# Create plot with examples of each method in each dataset

In [ ]:
# CBF: 1, 2, 10, 18, 19
# chinatown: 3, 7, 19
# coffee: 1, 3, 9, 16
# ECG200: 0, 13
# gunpoint: 9, 10, 15 (pesimos en general)
dataset_samples = [('CBF', 2), ('chinatown', 19), ('coffee', 3), ('ECG200', 13), ('gunpoint', 15)]
methods_trad_dict = {"wcf_ng": "W-CF", "ng": "NG", "ab_cf": "AB-CF", "subspace": "Sub-SpaCE"}
methods = list(methods_trad_dict.keys())
include_nun_list = [False, False, False, False]

In [ ]:
from collections import OrderedDict

# Define outer grid
fig = plt.figure(figsize=(16, 6))
grid_rows = len(methods)
grid_columns = len(dataset_samples)
outer_grid = gridspec.GridSpec(grid_rows, grid_columns, wspace=0, hspace=0)

# Iterate through outer grid plots
for i_outer, outer in enumerate(outer_grid):
    i_dataset = i_outer % grid_columns
    dataset, instance = dataset_samples[i_dataset]
    column_title = f'{dataset} instance {instance}'
    
    X_train, y_train, X_test, y_test = data_dict[dataset]
    nuns_idx = nuns_idx_dict[dataset]
    sol_cfs_methods = sol_cfs_dict[dataset]
    sol_cfs_methods = {method: sol_cfs_methods[method][instance]['cf'] for method in methods}

    # Calculate auxiliary variables
    method_names = list(sol_cfs_methods.keys())
    x_cfs = list(sol_cfs_methods.values())
    x_orig = X_test[instance]
    nun_example = X_train[nuns_idx[instance]]
    length = x_orig.shape[0]
    n_channels = x_orig.shape[1]

    # get method m
    m = i_outer // grid_columns
    try:
        # Get change mask 
        x_cf = x_cfs[m]
        method_name = method_names[m]
        found_counterfactual_mask = (x_orig != x_cf.reshape(1, length, n_channels)).astype(int)
        diff_mask = np.diff(found_counterfactual_mask, prepend=0, append=0, axis=1)
    except IndexError:
        break

    # Create inner grid
    inner_grid = gridspec.GridSpecFromSubplotSpec(n_channels, 1, wspace=0, hspace=0, subplot_spec=outer)
    sub_channel_axs = []
    for i in range(n_channels):
        ax = plt.Subplot(fig, inner_grid[i])
        if include_nun_list[m]:
            ax.plot(nun_example.reshape(1, length, n_channels)[:, :, i].flatten(), color='grey')
        ax.plot(x_cf.reshape(1, length, n_channels)[:, :, i].flatten(), color='red')
        ax.plot(x_orig.reshape(1, length, n_channels)[:, :, i].flatten(), color="#332288")
        # ax.axis("off")
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Add fill to changes
        starts = np.clip( np.where(diff_mask[0, :, i].flatten() == 1)[0] - 1 , 0, length)
        ends = np.where(diff_mask[0, :, i].flatten() == -1)[0]
        # Iterate over the mask segments and fill them with red color
        for i in range(0, len(starts)):
            start_idx = starts[i]
            end_idx = ends[i]
            # plt.fill_between(comp_df.iloc[start_idx:end_idx].index, comp_df.min(), comp_df.max(), color='red', alpha=0.7)
            ax.axvspan(start_idx, end_idx, alpha=0.2, color='red')
        
        # Set column titles
        if m == 0:
            ax.set_title(column_title)
        if i_dataset == 0:
            ax.set_ylabel(methods_trad_dict[method_name], size='large')
        # Add ax to list of subplots
        fig.add_subplot(ax)
        sub_channel_axs.append(ax)

# Plot figure
plt.tight_layout()
"""fig.legend(
    sub_channel_axs,     # The line objects
    labels=['$x^n$', 'x', "x'"],   # The labels for each line
    loc='lower center',
    # loc='center right',
    prop={'size': 14},
    ncol=3
)"""
# plt.show()
# plt.savefig('./experiments/results/examples_cfs.png', dpi=300)


In [ ]:
from collections import OrderedDict

# Define outer grid
fig = plt.figure(figsize=(12, 12))
grid_rows = len(dataset_samples)
grid_columns = len(methods)
outer_grid = gridspec.GridSpec(grid_rows, grid_columns, wspace=0, hspace=0)

# Iterate through outer grid plots
for i_outer, outer in enumerate(outer_grid):
    i_dataset = i_outer // grid_columns
    dataset, instance = dataset_samples[i_dataset]
    
    X_train, y_train, X_test, y_test = data_dict[dataset]
    nuns_idx = nuns_idx_dict[dataset]
    sol_cfs_methods = sol_cfs_dict[dataset]
    sol_cfs_methods = {method: sol_cfs_methods[method][instance]['cf'] for method in methods}

    # Calculate auxiliary variables
    method_names = list(sol_cfs_methods.keys())
    x_cfs = list(sol_cfs_methods.values())
    x_orig = X_test[instance]
    nun_example = X_train[nuns_idx[instance]]
    length = x_orig.shape[0]
    n_channels = x_orig.shape[1]

    # get method m
    m = i_outer % grid_columns
    try:
        # Get change mask 
        x_cf = x_cfs[m]
        method_name = method_names[m]
        found_counterfactual_mask = (x_orig != x_cf.reshape(1, length, n_channels)).astype(int)
        diff_mask = np.diff(found_counterfactual_mask, prepend=0, append=0, axis=1)
    except IndexError:
        break
    
    # Create inner grid
    inner_grid = gridspec.GridSpecFromSubplotSpec(n_channels, 1, wspace=0, hspace=0, subplot_spec=outer)
    sub_channel_axs = []
    for i in range(n_channels):
        ax = plt.Subplot(fig, inner_grid[i])
        if include_nun_list[m]:
            ax.plot(nun_example.reshape(1, length, n_channels)[:, :, i].flatten(), color='grey')
        ax.plot(x_cf.reshape(1, length, n_channels)[:, :, i].flatten(), color='red')
        ax.plot(x_orig.reshape(1, length, n_channels)[:, :, i].flatten(), color="#332288")
        # ax.axis("off")
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Add fill to changes
        starts = np.clip( np.where(diff_mask[0, :, i].flatten() == 1)[0] - 1 , 0, length)
        ends = np.where(diff_mask[0, :, i].flatten() == -1)[0]
        # Iterate over the mask segments and fill them with red color
        for i in range(0, len(starts)):
            start_idx = starts[i]
            end_idx = ends[i]
            # plt.fill_between(comp_df.iloc[start_idx:end_idx].index, comp_df.min(), comp_df.max(), color='red', alpha=0.7)
            ax.axvspan(start_idx, end_idx, alpha=0.2, color='red')
        
        # Set column titles
        column_title = methods_trad_dict[method_name]
        # row_title = f'{dataset} \n instance {instance}'
        row_title = f'{dataset} [{instance}]'
        if m == 0:
            ax.set_ylabel(row_title, fontsize=16, rotation=90)
        if i_dataset == 0:
            ax.set_title(column_title, fontsize=18)
            
        # Add ax to list of subplots
        fig.add_subplot(ax)
        sub_channel_axs.append(ax)

# Plot figure
plt.tight_layout()
"""fig.legend(
    sub_channel_axs,     # The line objects
    labels=['$x^n$', 'x', "x'"],   # The labels for each line
    # loc='lower center',
    loc='center right',
    prop={'size': 18},
    ncol=1
)"""
# plt.show()
plt.savefig('./experiments/results/examples_cfs.png', dpi=300)
